In [18]:
class SVM:
  def __init__(self, kernel='linear', C=10000.0, max_iter=100000, degree=3, gamma=1):
    self.kernel = {'poly'  : lambda x,y: np.dot(x, y.T)**degree,
                   'rbf'   : lambda x,y: np.exp(-gamma*np.sum((y - x[:,np.newaxis])**2, axis=-1)),
                   'linear': lambda x,y: np.dot(x, y.T)}[kernel]
    self.C = C
    self.max_iter = max_iter

  def restrict_to_square(self, t, v0, u):
    t = (np.clip(v0 + t*u, 0, self.C) - v0)[1]/u[1]
    return (np.clip(v0 + t*u, 0, self.C) - v0)[0]/u[0]

  def fit(self, X, y):
    self.X = X.copy()
    self.y = y * 2 - 1
    self.lambdas = np.zeros_like(self.y, dtype=float) #(rows,1)
    self.K = self.kernel(self.X, self.X) * self.y[:,np.newaxis] * self.y
    
    for _ in tqdm(range(self.max_iter)):
      for idxM in range(len(self.lambdas)):
        idxL = np.random.randint(0, len(self.lambdas))
        Q = self.K[[[idxM, idxM], [idxL, idxL]], [[idxM, idxL], [idxM, idxL]]]
        v0 = self.lambdas[[idxM, idxL]]
        k0 = 1 - np.sum(self.lambdas * self.K[[idxM, idxL]], axis=1)
        u = np.array([-self.y[idxL], self.y[idxM]])
        t_max = np.dot(k0, u) / (np.dot(np.dot(Q, u), u) + 1E-15)
        self.lambdas[[idxM, idxL]] = v0 + u * self.restrict_to_square(t_max, v0, u)
    
    idx, = np.nonzero(self.lambdas > 1E-15)
    self.b = np.mean((1.0 - np.sum(self.K[idx] * self.lambdas, axis=1)) * self.y[idx])
  
  def decision_function(self, X):
    return np.sum(self.kernel(X, self.X) * self.y * self.lambdas, axis=1) + self.b

  def predict(self, X):
    return (np.sign(self.decision_function(X)) + 1) // 2

# Scartch 

In [1]:
import pandas as pd 
import numpy as np 
import torch 
import matplotlib.pyplot as plt 
import sklearn 
import seaborn as sns 
import os 
from tqdm import tqdm 
from glob import glob 
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.manifold import MDS

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 데이터 로드 

In [7]:
from sklearn.datasets import load_digits,load_wine
#데이터 로드 
data = load_wine() 
df = pd.DataFrame(data.data)
df.columns = data['feature_names']
df['class'] = data['target']
df = df.sample(frac=1,random_state=42).reset_index(drop=True) #shuffle 

## 데이터 전처리 

In [8]:
# 결측치 확인 
df.isnull().sum().sum()

0

In [9]:
#정규화 
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
normalized_df = pd.DataFrame(normalizer.fit_transform(df))
normalized_df.columns = df.columns
normalized_df['class'] = df['class']

In [11]:
#Train - Test 분리 
n_train = int(len(df)*0.8)
train_x = np.array(df.iloc[:n_train].drop(columns='class'))
train_y = np.array(df.iloc[:n_train]['class'])

valid_x = np.array(df.iloc[n_train:].drop(columns='class'))
valid_y = np.array(df.iloc[n_train:]['class'])

## 모델 
- 커널 함수로 rbf 커널 함수 사용 

$K(x,y) = exp(-{||x-y||^2 \over 2\sigma^2})$

In [ ]:
#하이퍼 파라미터 설정 
class cfg:
    c = 10000
    max_iter = 100000
    gamma =1 

#커널 함수 정의 
def kernel(x,y):
    gamma = 1 
    return np.exp(-gamma*np.sum((y - x[:,np.newaxis])**2, axis=-1))

def restrict_to_square(t, v0, u):
    C = 10000
    t = (np.clip(v0 + t*u, 0, C) - v0)[1]/u[1]
    return (np.clip(v0 + t*u, 0, C) - v0)[0]/u[0]

# 모델 학습 
x = train_x 
y = train_y 

weight = np.zeros_like(y,dtype=float) #찾고자 하는 function의 
k = kernel(x,x) * y[:,np.newaxis]*y  #내적값을 구하기 위해 커널값 사용 

for i in tqdm(range(cfg.max_iter)):
    for idxM in range(len(lambdas)):
        idxL = np.random.randint(0,len(lambdas))

        Q = k[[[idxM,idxM],[idxL,idxL]],[[idxM,idxL],[idxM,idxL]]]

        v0 = lambdas[[idxM,idxL]]

        k0 = 1-np.sum(lambdas * k[[idxM,idxL]],axis=1)

        u = np.array([-y[idxL],y[idxM]])

        t_max = np.dot(k0,u) / (np.dot(np.dot(Q,u), u) + 1e-15)
        
        lambdas[[idxM,idxL]] = v0 + u * restrict_to_square(t_max,v0,u)

idx, = np.nonzero(lambdas > 1E-15)
b = np.mean((1.0 - np.sum(K[idx] * lambdas, axis=1)) * y[idx]) 

In [32]:
np.unique(train_y*2-1)

array([-1,  1,  3])

In [19]:
svm = SVM()
svm.fit(train_x,train_y)

  0%|          | 409/100000 [00:05<21:28, 77.30it/s]


KeyboardInterrupt: 

In [29]:
from sklearn.svm import SVC

svm  = SVC()
svm.fit(train_x,train_y)


SVC()